In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from causalml.inference.meta import BaseXRegressor, BaseRRegressor, BaseSRegressor, BaseTRegressor, BaseDRRegressor
from causalml.inference.tree import CausalRandomForestRegressor
from causalml.metrics import get_cumgain, auuc_score, plot_gain

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.utils.extmath import cartesian
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import xgboost as xgb

: 

In [ ]:
from copy import deepcopy
import logging
from tqdm import tqdm
from scipy.stats import norm
from scipy.spatial import cKDTree

from causalml.inference.meta.base import BaseLearner
from causalml.inference.meta.utils import (
    check_treatment_vector,
    check_p_conditions,
    convert_pd_to_np,
)
from causalml.metrics import regression_metrics
from causalml.propensity import compute_propensity_score

logger = logging.getLogger("causalml")

In [ ]:
class BaseHLearner(BaseLearner):
    """A parent class for H-learner regressor classes.

    A H-learner estimates treatment effects with three machine learning models."""

    def __init__(
        self,
        learner=None,
        outcome_learner=None,
        control_effect_learner=None,
        treatment_effect_learner=None,
        ate_alpha=0.05,
        control_name=0,
    ):
        """Initialize a H-learner.

        Args:
            learner (optional): a model to estimate outcomes and treatment effects in both the control and treatment
                groups
            outcome_learner (optional): a model to estimate outcomes
            control_effect_learner (optional): a model to estimate treatment effects in the control group
            treatment_effect_learner (optional): a model to estimate treatment effects in the treatment group
            ate_alpha (float, optional): the confidence level alpha of the ATE estimate
            control_name (str or int, optional): name of control group
        """
        assert (learner is not None) or (
            (outcome_learner is not None)
            and (control_effect_learner is not None)
            and (treatment_effect_learner is not None)
        )

        if outcome_learner is None:
            self.model_mu = deepcopy(learner)
        else:
            self.model_mu = outcome_learner

        if control_effect_learner is None:
            self.model_tau_c = deepcopy(learner)
        else:
            self.model_tau_c = control_effect_learner

        if treatment_effect_learner is None:
            self.model_tau_t = deepcopy(learner)
        else:
            self.model_tau_t = treatment_effect_learner

        self.ate_alpha = ate_alpha
        self.control_name = control_name

        self.propensity = None
        self.propensity_model = None
        self.model_p = LogisticRegression()

    def __repr__(self):
        return (
            "{}(outcome_learner={},\n"
            "\tcontrol_effect_learner={},\n"
            "\ttreatment_effect_learner={})".format(
                self.__class__.__name__,
                self.model_mu.__repr__(),
                self.model_tau_c.__repr__(),
                self.model_tau_t.__repr__(),
            )
        )

    def fit(self, X, treatment, y, p=None):
        """Fit the inference model.

        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series): a treatment vector
            y (np.array or pd.Series): an outcome vector
            p (np.ndarray or pd.Series or dict, optional): an array of propensity scores of float (0,1) in the
                single-treatment case; or, a dictionary of treatment groups that map to propensity vectors of
                float (0,1); if None will run ElasticNetPropensityModel() to generate the propensity scores.
        """
        X, treatment, y = convert_pd_to_np(X, treatment, y)
        check_treatment_vector(treatment, self.control_name)
        self.t_groups = np.unique(treatment[treatment != self.control_name])
        self.t_groups.sort()

        if p is None:
            self._set_propensity_models(X=X, treatment=treatment, y=y)
            p = self.propensity
        else:
            p = self._format_p(p, self.t_groups)

        self._classes = {group: i for i, group in enumerate(self.t_groups)}
        self.models_tau_c = {
            group: deepcopy(self.model_tau_c) for group in self.t_groups
        }
        self.models_tau_t = {
            group: deepcopy(self.model_tau_t) for group in self.t_groups
        }

        self.vars_c = {}
        self.vars_t = {}

        # Train outcome model
        self.model_mu.fit(X, y)

        for group in self.t_groups:
            mask = (treatment == group) | (treatment == self.control_name)
            treatment_filt = treatment[mask]
            X_filt = X[mask]
            y_filt = y[mask]
            w = (treatment_filt == group).astype(int)

            # Calculate variances and treatment effects
            var_c = (
                y_filt[w == 0] - self.model_mu.predict(X_filt[w == 0])
            ).var()
            self.vars_c[group] = var_c
            var_t = (
                y_filt[w == 1] - self.model_mu.predict(X_filt[w == 1])
            ).var()
            self.vars_t[group] = var_t

            # Train treatment models
            d_c = (self.model_mu.predict(X_filt[w == 0]) - y_filt[w == 0])
            d_t = (y_filt[w == 1] - self.model_mu.predict(X_filt[w == 1]))
            self.models_tau_c[group].fit(X_filt[w == 0], d_c)
            self.models_tau_t[group].fit(X_filt[w == 1], d_t)

    def predict(
        self, X, treatment=None, y=None, p=None, return_components=False, verbose=True
    ):
        """Predict treatment effects.

        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series, optional): a treatment vector
            y (np.array or pd.Series, optional): an outcome vector
            p (np.ndarray or pd.Series or dict, optional): an array of propensity scores of float (0,1) in the
                single-treatment case; or, a dictionary of treatment groups that map to propensity vectors of
                float (0,1); if None will run ElasticNetPropensityModel() to generate the propensity scores.
            return_components (bool, optional): whether to return differences for treatment and control seperately
            verbose (bool, optional): whether to output progress logs
        Returns:
            (numpy.ndarray): Predictions of treatment effects.
        """
        X, treatment, y = convert_pd_to_np(X, treatment, y)

        if p is None:
            logger.info("Generating propensity score")
            p = dict()
            for group in self.t_groups:
                p_model = self.propensity_model[group]
                p[group] = p_model.predict(X)
        else:
            p = self._format_p(p, self.t_groups)

        te = np.zeros((X.shape[0], self.t_groups.shape[0]))
        dhat_cs = {}
        dhat_ts = {}

        for i, group in enumerate(self.t_groups):
            model_tau_c = self.models_tau_c[group]
            model_tau_t = self.models_tau_t[group]
            dhat_cs[group] = model_tau_c.predict(X)
            dhat_ts[group] = model_tau_t.predict(X)

            _te = (dhat_cs[group] + dhat_ts[group]).reshape(
                -1, 1
            )
            te[:, i] = np.ravel(_te)

            if (y is not None) and (treatment is not None) and verbose:
                mask = (treatment == group) | (treatment == self.control_name)
                treatment_filt = treatment[mask]
                X_filt = X[mask]
                y_filt = y[mask]
                w = (treatment_filt == group).astype(int)

                yhat = np.zeros_like(y, dtype=float)
                yhat = self.model_mu.predict(X)

                logger.info("Error metrics for group {}".format(group))
                regression_metrics(y, yhat, w)

        if not return_components:
            return te
        else:
            return te, dhat_cs, dhat_ts

    def fit_predict(
        self,
        X,
        treatment,
        y,
        p=None,
        return_ci=False,
        n_bootstraps=1000,
        bootstrap_size=10000,
        return_components=False,
        verbose=True,
    ):
        """Fit the treatment effect and outcome models of the R learner and predict treatment effects.

        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series): a treatment vector
            y (np.array or pd.Series): an outcome vector
            p (np.ndarray or pd.Series or dict, optional): an array of propensity scores of float (0,1) in the
                single-treatment case; or, a dictionary of treatment groups that map to propensity vectors of
                float (0,1); if None will run ElasticNetPropensityModel() to generate the propensity scores.
            return_ci (bool): whether to return confidence intervals
            n_bootstraps (int): number of bootstrap iterations
            bootstrap_size (int): number of samples per bootstrap
            return_components (bool, optional): whether to return outcome for treatment and control seperately
            verbose (str): whether to output progress logs
        Returns:
            (numpy.ndarray): Predictions of treatment effects. Output dim: [n_samples, n_treatment]
                If return_ci, returns CATE [n_samples, n_treatment], LB [n_samples, n_treatment],
                UB [n_samples, n_treatment]
        """
        X, treatment, y = convert_pd_to_np(X, treatment, y)
        self.fit(X, treatment, y, p)

        if p is None:
            p = self.propensity
        else:
            p = self._format_p(p, self.t_groups)

        te = self.predict(
            X, treatment=treatment, y=y, p=p, return_components=return_components
        )

        if not return_ci:
            return te
        else:
            t_groups_global = self.t_groups
            _classes_global = self._classes
            model_mu_global = deepcopy(self.model_mu)
            models_tau_c_global = deepcopy(self.models_tau_c)
            models_tau_t_global = deepcopy(self.models_tau_t)
            te_bootstraps = np.zeros(
                shape=(X.shape[0], self.t_groups.shape[0], n_bootstraps)
            )

            logger.info("Bootstrap Confidence Intervals")
            for i in tqdm(range(n_bootstraps)):
                te_b = self.bootstrap(X, treatment, y, p, size=bootstrap_size)
                te_bootstraps[:, :, i] = te_b

            te_lower = np.percentile(te_bootstraps, (self.ate_alpha / 2) * 100, axis=2)
            te_upper = np.percentile(
                te_bootstraps, (1 - self.ate_alpha / 2) * 100, axis=2
            )

            # set member variables back to global (currently last bootstrapped outcome)
            self.t_groups = t_groups_global
            self._classes = _classes_global
            self.model_mu = deepcopy(model_mu_global)
            self.models_tau_c = deepcopy(models_tau_c_global)
            self.models_tau_t = deepcopy(models_tau_t_global)

            return (te, te_lower, te_upper)

    def estimate_ate(
        self,
        X,
        treatment,
        y,
        p=None,
        bootstrap_ci=False,
        n_bootstraps=1000,
        bootstrap_size=10000,
        pretrain=False,
    ):
        """Estimate the Average Treatment Effect (ATE).

        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series): a treatment vector
            y (np.array or pd.Series): an outcome vector
            p (np.ndarray or pd.Series or dict, optional): an array of propensity scores of float (0,1) in the
                single-treatment case; or, a dictionary of treatment groups that map to propensity vectors of
                float (0,1); if None will run ElasticNetPropensityModel() to generate the propensity scores.
            bootstrap_ci (bool): whether run bootstrap for confidence intervals
            n_bootstraps (int): number of bootstrap iterations
            bootstrap_size (int): number of samples per bootstrap
            pretrain (bool): whether a model has been fit, default False.
        Returns:
            The mean and confidence interval (LB, UB) of the ATE estimate.
        """
        if pretrain:
            if p is None:
                # when p is null, use pretrain propensity score
                if not self.propensity:
                    raise ValueError("no propensity score, please call fit() first")
                te, dhat_cs, dhat_ts = self.predict(
                    X, treatment, y, p=self.propensity, return_components=True
                )
            else:
                p = self._format_p(p, self.t_groups)
                te, dhat_cs, dhat_ts = self.predict(
                    X, treatment, y, p=p, return_components=True
                )
        else:
            te, dhat_cs, dhat_ts = self.fit_predict(
                X, treatment, y, p, return_components=True
            )
        X, treatment, y = convert_pd_to_np(X, treatment, y)

        if p is None:
            p = self.propensity
        else:
            p = self._format_p(p, self.t_groups)

        ate = np.zeros(self.t_groups.shape[0])
        ate_lb = np.zeros(self.t_groups.shape[0])
        ate_ub = np.zeros(self.t_groups.shape[0])

        for i, group in enumerate(self.t_groups):
            _ate = te[:, i].mean()

            mask = (treatment == group) | (treatment == self.control_name)
            treatment_filt = treatment[mask]
            w = (treatment_filt == group).astype(int)
            prob_treatment = float(sum(w)) / w.shape[0]

            dhat_c = dhat_cs[group][mask]
            dhat_t = dhat_ts[group][mask]
            p_filt = p[group][mask]

            # SE formula is based on the lower bound formula (7) from Imbens, Guido W., and Jeffrey M. Wooldridge. 2009.
            # "Recent Developments in the Econometrics of Program Evaluation." Journal of Economic Literature
            se = np.sqrt(
                (
                    self.vars_t[group] / prob_treatment
                    + self.vars_c[group] / (1 - prob_treatment)
                    + (p_filt * dhat_c + (1 - p_filt) * dhat_t).var()
                )
                / w.shape[0]
            )

            _ate_lb = _ate - se * norm.ppf(1 - self.ate_alpha / 2)
            _ate_ub = _ate + se * norm.ppf(1 - self.ate_alpha / 2)

            ate[i] = _ate
            ate_lb[i] = _ate_lb
            ate_ub[i] = _ate_ub

        if not bootstrap_ci:
            return ate, ate_lb, ate_ub
        else:
            t_groups_global = self.t_groups
            _classes_global = self._classes
            model_mu_global = deepcopy(self.model_mu)
            models_tau_c_global = deepcopy(self.models_tau_c)
            models_tau_t_global = deepcopy(self.models_tau_t)

            logger.info("Bootstrap Confidence Intervals for ATE")
            ate_bootstraps = np.zeros(shape=(self.t_groups.shape[0], n_bootstraps))

            for n in tqdm(range(n_bootstraps)):
                cate_b = self.bootstrap(X, treatment, y, p, size=bootstrap_size)
                ate_bootstraps[:, n] = cate_b.mean()

            ate_lower = np.percentile(
                ate_bootstraps, (self.ate_alpha / 2) * 100, axis=1
            )
            ate_upper = np.percentile(
                ate_bootstraps, (1 - self.ate_alpha / 2) * 100, axis=1
            )

            # set member variables back to global (currently last bootstrapped outcome)
            self.t_groups = t_groups_global
            self._classes = _classes_global
            self.model_mu = deepcopy(model_mu_global)
            self.models_tau_c = deepcopy(models_tau_c_global)
            self.models_tau_t = deepcopy(models_tau_t_global)
            return ate, ate_lower, ate_upper

In [ ]:
class BaseHRegressor(BaseHLearner):
    """
    A parent class for H-learner regressor classes.
    """

    def __init__(
        self,
        learner=None,
        outcome_learner=None,
        control_effect_learner=None,
        treatment_effect_learner=None,
        ate_alpha=0.05,
        control_name=0,
    ):
        """Initialize an X-learner regressor.

        Args:
            learner (optional): a model to estimate outcomes and treatment effects in both the control and treatment
                groups
            outcome_learner (optional): a model to estimate outcomes
            control_effect_learner (optional): a model to estimate treatment effects in the control group
            treatment_effect_learner (optional): a model to estimate treatment effects in the treatment group
            ate_alpha (float, optional): the confidence level alpha of the ATE estimate
            control_name (str or int, optional): name of control group
        """
        super().__init__(
            learner=learner,
            outcome_learner=outcome_learner,
            control_effect_learner=control_effect_learner,
            treatment_effect_learner=treatment_effect_learner,
            ate_alpha=ate_alpha,
            control_name=control_name,
        )

In [ ]:
class BasePLearner(BaseLearner):
    """A parent class for P-learner regressor classes.

    A P-learner estimates treatment effects with one machine learning model."""

    def __init__(
        self,
        learner=None,
        ate_alpha=0.05,
        control_name=0,
    ):
        """Initialize a P-learner.

        Args:
            learner (optional): a model to estimate outcomes and treatment effects in both the control and treatment
                groups
            outcome_learner (optional): a model to estimate outcomes
            control_effect_learner (optional): a model to estimate treatment effects in the control group
            treatment_effect_learner (optional): a model to estimate treatment effects in the treatment group
            ate_alpha (float, optional): the confidence level alpha of the ATE estimate
            control_name (str or int, optional): name of control group
        """
        assert (learner is not None)

        self.model_nu = deepcopy(learner)

        self.ate_alpha = ate_alpha
        self.control_name = control_name

        self.propensity = None
        self.propensity_model = None
        self.model_p = LogisticRegression()

    def __repr__(self):
        return (
            "{}(pairwise_learner={})".format(
                self.__class__.__name__,
                self.model_nu.__repr__(),
            )
        )
    
    def pair_data(self, X, treatment, y, num_samples):

        # Split T=0 and T=1
        X0 = X[treatment == 0]
        Y0 = y[treatment == 0]
        X1 = X[treatment == 1]
        Y1 = y[treatment == 1]

        scaler = MinMaxScaler()
        X1_scaled = scaler.fit_transform(X1)
        X0_scaled = scaler.fit_transform(X0)

        # Build KD-tree from Y
        tree = cKDTree(X1_scaled)

        # Query for k nearest neighbours
        distances, indices = tree.query(X0_scaled, k=num_samples)
        # Randomly sample rows from X_treated for each row in X_control
        random_indices = np.random.randint(0, len(X1), (len(X0), num_samples))
        
        # Expand T=0 data to match shape
        X0_expanded = np.repeat(X0, num_samples, axis=0)  # Repeat X0 for 5 times
        Y0_expanded = np.repeat(Y0, num_samples, axis=0)  # Repeat Y0 for 5 times

        # Get corresponding T=1 data (using the random indices)
        X1_sampled = X1[indices.flatten()]
        Y1_sampled = Y1[indices.flatten()]

        # Reshape the sampled X1 and Y1 for the final DataFrame
        X1_sampled = X1_sampled.reshape(-1, X.shape[1])
        Y1_sampled = Y1_sampled.reshape(-1, 1)

        # Combine the matrices (X0 with X1, Y0 with Y1)
        X_combined = np.concatenate([X0_expanded, X1_sampled], axis=1)  # Concatenate feature matrices
        Y_combined = np.column_stack((Y0_expanded, Y1_sampled))  # Concatenate outcome matrices

        # Calculate outcome differences
        nu = Y_combined[:,1] - Y_combined[:,0]

        return X_combined, nu

    def fit(self, X, treatment, y, p, num_samples=10):
        """Fit the inference model.

        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series): a treatment vector
            y (np.array or pd.Series): an outcome vector
            p (np.ndarray or pd.Series or dict, optional): an array of propensity scores of float (0,1) in the
                single-treatment case; or, a dictionary of treatment groups that map to propensity vectors of
                float (0,1); if None will run ElasticNetPropensityModel() to generate the propensity scores.
            num_samples (optional): number of treated samples to pair with each control.
        """
        
        X, y = convert_pd_to_np(X, y)
        X_pair, nu = self.pair_data(X, treatment, y, num_samples)

        self.t_groups = np.unique(treatment[treatment != self.control_name])
        self.t_groups.sort()

        self._classes = {group: i for i, group in enumerate(self.t_groups)}

        self.vars_c = {}
        self.vars_t = {}

        # Train outcome model
        self.model_nu.fit(X_pair, nu)

        # for group in self.t_groups:
        #     mask = (treatment == group) | (treatment == self.control_name)
        #     treatment_filt = treatment[mask]
        #     X_filt = X[mask]
        #     y_filt = y[mask]
        #     w = (treatment_filt == group).astype(int)

        #     # Calculate variances and treatment effects
        #     var_c = (
        #         y_filt[w == 0] - self.model_mu.predict(X_filt[w == 0])
        #     ).var()
        #     self.vars_c[group] = var_c
        #     var_t = (
        #         y_filt[w == 1] - self.model_mu.predict(X_filt[w == 1])
        #     ).var()
        #     self.vars_t[group] = var_t

        #     # Train treatment models
        #     d_c = (self.model_mu.predict(X_filt[w == 0]) - y_filt[w == 0])
        #     d_t = (y_filt[w == 1] - self.model_mu.predict(X_filt[w == 1]))
        #     self.models_tau_c[group].fit(X_filt[w == 0], d_c)
        #     self.models_tau_t[group].fit(X_filt[w == 1], d_t)

    def predict(
        self, X, treatment=None, y=None, p=None, return_components=False, verbose=True
    ):
        """Predict treatment effects.

        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series, optional): a treatment vector
            y (np.array or pd.Series, optional): an outcome vector
            p (np.ndarray or pd.Series or dict, optional): an array of propensity scores of float (0,1) in the
                single-treatment case; or, a dictionary of treatment groups that map to propensity vectors of
                float (0,1); if None will run ElasticNetPropensityModel() to generate the propensity scores.
            return_components (bool, optional): whether to return differences for treatment and control seperately
            verbose (bool, optional): whether to output progress logs
        Returns:
            (numpy.ndarray): Predictions of treatment effects.
        """

        X, treatment, y = convert_pd_to_np(X, treatment, y)
        X = np.concatenate([X, X], axis=1)

        # if p is None:
        #     logger.info("Generating propensity score")
        #     p = dict()
        #     for group in self.t_groups:
        #         p_model = self.propensity_model[group]
        #         p[group] = p_model.predict(X)
        # else:
        #     p = self._format_p(p, self.t_groups)

        te = np.zeros((X.shape[0], self.t_groups.shape[0]))
        dhat_cs = {}
        dhat_ts = {}

        _te = (self.model_nu.predict(X)).reshape(
            -1, 1
        )
        te[:, 0] = np.ravel(_te)

        # for i, group in enumerate(self.t_groups):
        #     model_tau_c = self.models_tau_c[group]
        #     model_tau_t = self.models_tau_t[group]
        #     dhat_cs[group] = model_tau_c.predict(X)
        #     dhat_ts[group] = model_tau_t.predict(X)

        #     _te = (dhat_cs[group] + dhat_ts[group]).reshape(
        #         -1, 1
        #     )
        #     te[:, i] = np.ravel(_te)

        #     if (y is not None) and (treatment is not None) and verbose:
        #         mask = (treatment == group) | (treatment == self.control_name)
        #         treatment_filt = treatment[mask]
        #         X_filt = X[mask]
        #         y_filt = y[mask]
        #         w = (treatment_filt == group).astype(int)

        #         yhat = np.zeros_like(y, dtype=float)
        #         yhat = self.model_mu.predict(X)

        #         logger.info("Error metrics for group {}".format(group))
        #         regression_metrics(y, yhat, w)

        if not return_components:
            return te
        else:
            return te, dhat_cs, dhat_ts

    def fit_predict(
        self,
        X,
        treatment,
        y,
        p=None,
        num_samples=10,
        return_ci=False,
        n_bootstraps=1000,
        bootstrap_size=10000,
        return_components=False,
        verbose=True,
    ):
        """Fit the treatment effect and outcome models of the R learner and predict treatment effects.

        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series): a treatment vector
            y (np.array or pd.Series): an outcome vector
            p (np.ndarray or pd.Series or dict, optional): an array of propensity scores of float (0,1) in the
                single-treatment case; or, a dictionary of treatment groups that map to propensity vectors of
                float (0,1); if None will run ElasticNetPropensityModel() to generate the propensity scores.
            return_ci (bool): whether to return confidence intervals
            n_bootstraps (int): number of bootstrap iterations
            bootstrap_size (int): number of samples per bootstrap
            return_components (bool, optional): whether to return outcome for treatment and control seperately
            verbose (str): whether to output progress logs
        Returns:
            (numpy.ndarray): Predictions of treatment effects. Output dim: [n_samples, n_treatment]
                If return_ci, returns CATE [n_samples, n_treatment], LB [n_samples, n_treatment],
                UB [n_samples, n_treatment]
        """
        X, treatment, y = convert_pd_to_np(X, treatment, y)
        self.fit(X, treatment, y, p, num_samples)

        if p is None:
            p = self.propensity
        else:
            p = self._format_p(p, self.t_groups)

        te = self.predict(
            X, treatment=treatment, y=y, p=p, return_components=return_components
        )

        if not return_ci:
            return te
        else:
            t_groups_global = self.t_groups
            _classes_global = self._classes
            model_nu_global = deepcopy(self.model_nu)
            te_bootstraps = np.zeros(
                shape=(X.shape[0], self.t_groups.shape[0], n_bootstraps)
            )

            logger.info("Bootstrap Confidence Intervals")
            for i in tqdm(range(n_bootstraps)):
                te_b = self.bootstrap(X, treatment, y, p, size=bootstrap_size)
                te_bootstraps[:, :, i] = te_b

            te_lower = np.percentile(te_bootstraps, (self.ate_alpha / 2) * 100, axis=2)
            te_upper = np.percentile(
                te_bootstraps, (1 - self.ate_alpha / 2) * 100, axis=2
            )

            # set member variables back to global (currently last bootstrapped outcome)
            self.t_groups = t_groups_global
            self._classes = _classes_global
            self.model_nu = deepcopy(model_nu_global)

            return (te, te_lower, te_upper)

    def estimate_ate(
        self,
        X,
        treatment,
        y,
        p=None,
        bootstrap_ci=False,
        n_bootstraps=1000,
        bootstrap_size=10000,
        pretrain=False,
    ):
        """Estimate the Average Treatment Effect (ATE).

        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series): a treatment vector
            y (np.array or pd.Series): an outcome vector
            p (np.ndarray or pd.Series or dict, optional): an array of propensity scores of float (0,1) in the
                single-treatment case; or, a dictionary of treatment groups that map to propensity vectors of
                float (0,1); if None will run ElasticNetPropensityModel() to generate the propensity scores.
            bootstrap_ci (bool): whether run bootstrap for confidence intervals
            n_bootstraps (int): number of bootstrap iterations
            bootstrap_size (int): number of samples per bootstrap
            pretrain (bool): whether a model has been fit, default False.
        Returns:
            The mean and confidence interval (LB, UB) of the ATE estimate.
        """
        if pretrain:
            if p is None:
                # when p is null, use pretrain propensity score
                if not self.propensity:
                    raise ValueError("no propensity score, please call fit() first")
                te, dhat_cs, dhat_ts = self.predict(
                    X, treatment, y, p=self.propensity, return_components=True
                )
            else:
                p = self._format_p(p, self.t_groups)
                te, dhat_cs, dhat_ts = self.predict(
                    X, treatment, y, p=p, return_components=True
                )
        else:
            te, dhat_cs, dhat_ts = self.fit_predict(
                X, treatment, y, p, return_components=True
            )
        X, treatment, y = convert_pd_to_np(X, treatment, y)

        if p is None:
            p = self.propensity
        else:
            p = self._format_p(p, self.t_groups)

        ate = np.zeros(self.t_groups.shape[0])
        ate_lb = np.zeros(self.t_groups.shape[0])
        ate_ub = np.zeros(self.t_groups.shape[0])

        ate[0] = te[:, 0].mean()

        # for i, group in enumerate(self.t_groups):
        #     _ate = te[:, i].mean()

        #     mask = (treatment == group) | (treatment == self.control_name)
        #     treatment_filt = treatment[mask]
        #     w = (treatment_filt == group).astype(int)
        #     prob_treatment = float(sum(w)) / w.shape[0]

        #     dhat_c = dhat_cs[group][mask]
        #     dhat_t = dhat_ts[group][mask]
        #     p_filt = p[group][mask]

        #     # SE formula is based on the lower bound formula (7) from Imbens, Guido W., and Jeffrey M. Wooldridge. 2009.
        #     # "Recent Developments in the Econometrics of Program Evaluation." Journal of Economic Literature
        #     se = np.sqrt(
        #         (
        #             self.vars_t[group] / prob_treatment
        #             + self.vars_c[group] / (1 - prob_treatment)
        #             + (p_filt * dhat_c + (1 - p_filt) * dhat_t).var()
        #         )
        #         / w.shape[0]
        #     )

        #     _ate_lb = _ate - se * norm.ppf(1 - self.ate_alpha / 2)
        #     _ate_ub = _ate + se * norm.ppf(1 - self.ate_alpha / 2)

        #     ate[i] = _ate
        #     ate_lb[i] = _ate_lb
        #     ate_ub[i] = _ate_ub

        if not bootstrap_ci:
            return ate, ate_lb, ate_ub
        else:
            t_groups_global = self.t_groups
            _classes_global = self._classes
            model_nu_global = deepcopy(self.model_nu)

            logger.info("Bootstrap Confidence Intervals for ATE")
            ate_bootstraps = np.zeros(shape=(self.t_groups.shape[0], n_bootstraps))

            for n in tqdm(range(n_bootstraps)):
                cate_b = self.bootstrap(X, treatment, y, p, size=bootstrap_size)
                ate_bootstraps[:, n] = cate_b.mean()

            ate_lower = np.percentile(
                ate_bootstraps, (self.ate_alpha / 2) * 100, axis=1
            )
            ate_upper = np.percentile(
                ate_bootstraps, (1 - self.ate_alpha / 2) * 100, axis=1
            )

            # set member variables back to global (currently last bootstrapped outcome)
            self.t_groups = t_groups_global
            self._classes = _classes_global
            self.model_nu = deepcopy(model_nu_global)
            return ate, ate_lower, ate_upper

In [ ]:
class BasePRegressor(BasePLearner):
    """
    A parent class for P-learner regressor classes.
    """

    def __init__(
        self,
        learner=None,
        ate_alpha=0.05,
        control_name=0,
    ):
        """Initialize an X-learner regressor.

        Args:
            learner (optional): a model to estimate outcomes and treatment effects in both the control and treatment
                groups
            ate_alpha (float, optional): the confidence level alpha of the ATE estimate
            control_name (str or int, optional): name of control group
        """
        super().__init__(
            learner=learner,
            ate_alpha=ate_alpha,
            control_name=control_name,
        )

In [ ]:
class BasePHLearner(BaseLearner):
    """A parent class for PH-learner regressor classes.

    A PH-learner estimates treatment effects with two machine learning models."""

    def __init__(
        self,
        learner=None,
        outcome_learner=None,
        pair_learner=None,
        ate_alpha=0.05,
        control_name=0,
    ):
        """Initialize a H-learner.

        Args:
            learner (optional): a model to estimate outcomes and treatment effects in both the control and treatment
                groups
            outcome_learner (optional): a model to estimate outcomes
            pair_learner (optional): a model to estimate pair outcome differences
            treatment_effect_learner (optional): a model to estimate treatment effects in the treatment group
            ate_alpha (float, optional): the confidence level alpha of the ATE estimate
            control_name (str or int, optional): name of control group
        """
        assert (learner is not None) or (
            (outcome_learner is not None)
            and (pair_learner is not None)
        )

        if outcome_learner is None:
            self.model_mu = deepcopy(learner)
        else:
            self.model_mu = outcome_learner

        if pair_learner is None:
            self.model_nu = deepcopy(learner)
        else:
            self.model_nu = pair_learner

        self.ate_alpha = ate_alpha
        self.control_name = control_name

        self.propensity = None
        self.propensity_model = None
        self.model_p = LogisticRegression()

    def __repr__(self):
        return (
            "{}(outcome_learner={},\n"
            "\tpair_learner={})".format(
                self.__class__.__name__,
                self.model_mu.__repr__(),
                self.model_nu.__repr__()
            )
        )
    
    def pair_data(self, X, treatment, y, num_samples):

        # Split T=0 and T=1
        X0 = X[treatment == 0]
        Y0 = y[treatment == 0]
        X1 = X[treatment == 1]
        Y1 = y[treatment == 1]

        scaler = MinMaxScaler()
        X1_scaled = scaler.fit_transform(X1)
        X0_scaled = scaler.fit_transform(X0)

        # Build KD-tree from Y
        tree = cKDTree(X1_scaled)

        # Query for k nearest neighbours
        distances, indices = tree.query(X0_scaled, k=num_samples)
        # Randomly sample rows from X_treated for each row in X_control
        random_indices = np.random.randint(0, len(X1), (len(X0), num_samples))

        # Expand T=0 data to match shape
        X0_expanded = np.repeat(X0, num_samples, axis=0)  # Repeat X0 for 5 times
        Y0_expanded = np.repeat(Y0, num_samples, axis=0)  # Repeat Y0 for 5 times

        # Get corresponding T=1 data (using the random indices)
        X1_sampled = X1[indices.flatten()]
        Y1_sampled = Y1[indices.flatten()]

        # Reshape the sampled X1 and Y1 for the final DataFrame
        X1_sampled = X1_sampled.reshape(-1, X.shape[1])
        Y1_sampled = Y1_sampled.reshape(-1, 1)

        # Combine the matrices (X0 with X1, Y0 with Y1)
        X_combined = np.concatenate([X0_expanded, X1_sampled], axis=1)  # Concatenate feature matrices
        Y_combined = np.column_stack((Y0_expanded, Y1_sampled))  # Concatenate outcome matrices

        # Calculate outcome differences
        nu = Y_combined[:,1] - self.model_mu.predict(X1_sampled) + self.model_mu.predict(X0_expanded) - Y_combined[:,0]

        return X_combined, nu

    def fit(self, X, treatment, y, p=None, num_samples=5):
        """Fit the inference model.

        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series): a treatment vector
            y (np.array or pd.Series): an outcome vector
            p (np.ndarray or pd.Series or dict, optional): an array of propensity scores of float (0,1) in the
                single-treatment case; or, a dictionary of treatment groups that map to propensity vectors of
                float (0,1); if None will run ElasticNetPropensityModel() to generate the propensity scores.
            num_samples (int, optional): number of pairs for each control observation
        """
        X, treatment, y = convert_pd_to_np(X, treatment, y)
        check_treatment_vector(treatment, self.control_name)

        self.t_groups = np.unique(treatment[treatment != self.control_name])
        self.t_groups.sort()

        self._classes = {group: i for i, group in enumerate(self.t_groups)}

        # if p is None:
        #     self._set_propensity_models(X=X, treatment=treatment, y=y)
        #     p = self.propensity
        # else:
        #     p = self._format_p(p, self.t_groups)

        self.vars_c = {}
        self.vars_t = {}

        # Train outcome model
        self.model_mu.fit(X, y)

        # Train pair model
        X_paired, nu = self.pair_data(X, treatment, y, num_samples)
        self.model_nu.fit(X_paired, nu)

    def predict(
        self, X, treatment=None, y=None, p=None, return_components=False, verbose=True
    ):
        """Predict treatment effects.

        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series, optional): a treatment vector
            y (np.array or pd.Series, optional): an outcome vector
            p (np.ndarray or pd.Series or dict, optional): an array of propensity scores of float (0,1) in the
                single-treatment case; or, a dictionary of treatment groups that map to propensity vectors of
                float (0,1); if None will run ElasticNetPropensityModel() to generate the propensity scores.
            return_components (bool, optional): whether to return differences for treatment and control seperately
            verbose (bool, optional): whether to output progress logs
        Returns:
            (numpy.ndarray): Predictions of treatment effects.
        """
        X= convert_pd_to_np(X)
        X_paired = np.concatenate([X, X], axis=1)

        # if p is None:
        #     logger.info("Generating propensity score")
        #     p = dict()
        #     for group in self.t_groups:
        #         p_model = self.propensity_model[group]
        #         p[group] = p_model.predict(X)
        # else:
        #     p = self._format_p(p, self.t_groups)

        te = np.zeros((X.shape[0], self.t_groups.shape[0]))

        fhat = self.model_mu.predict(X)
        tauhat = self.model_nu.predict(X_paired)

        _te = (tauhat).reshape(
            -1, 1
        )
        te[:, 0] = np.ravel(_te)

        if not return_components:
            return te
        else:
            return te, fhat

    def fit_predict(
        self,
        X,
        treatment,
        y,
        p=None,
        num_samples=5,
        return_ci=False,
        n_bootstraps=1000,
        bootstrap_size=10000,
        return_components=False,
        verbose=True,
    ):
        """Fit the treatment effect and outcome models of the R learner and predict treatment effects.

        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series): a treatment vector
            y (np.array or pd.Series): an outcome vector
            p (np.ndarray or pd.Series or dict, optional): an array of propensity scores of float (0,1) in the
                single-treatment case; or, a dictionary of treatment groups that map to propensity vectors of
                float (0,1); if None will run ElasticNetPropensityModel() to generate the propensity scores.
            num_samples (int): number of pairs for each control observation
            return_ci (bool): whether to return confidence intervals
            n_bootstraps (int): number of bootstrap iterations
            bootstrap_size (int): number of samples per bootstrap
            return_components (bool, optional): whether to return outcome for treatment and control seperately
            verbose (str): whether to output progress logs
        Returns:
            (numpy.ndarray): Predictions of treatment effects. Output dim: [n_samples, n_treatment]
                If return_ci, returns CATE [n_samples, n_treatment], LB [n_samples, n_treatment],
                UB [n_samples, n_treatment]
        """
        X, treatment, y = convert_pd_to_np(X, treatment, y)
        self.fit(X, treatment, y, p, num_samples)

        # if p is None:
        #     p = self.propensity
        # else:
        #     p = self._format_p(p, self.t_groups)

        te = self.predict(
            X, return_components=return_components
        )

        if not return_ci:
            return te
        else:
            t_groups_global = self.t_groups
            _classes_global = self._classes
            model_mu_global = deepcopy(self.model_mu)
            model_nu_global = deepcopy(self.model_nu)
            te_bootstraps = np.zeros(
                shape=(X.shape[0], self.t_groups.shape[0], n_bootstraps)
            )

            logger.info("Bootstrap Confidence Intervals")
            for i in tqdm(range(n_bootstraps)):
                te_b = self.bootstrap(X, treatment, y, p, size=bootstrap_size)
                te_bootstraps[:, :, i] = te_b

            te_lower = np.percentile(te_bootstraps, (self.ate_alpha / 2) * 100, axis=2)
            te_upper = np.percentile(
                te_bootstraps, (1 - self.ate_alpha / 2) * 100, axis=2
            )

            # set member variables back to global (currently last bootstrapped outcome)
            self.t_groups = t_groups_global
            self._classes = _classes_global
            self.model_mu = deepcopy(model_mu_global)
            self.model_nu = deepcopy(model_nu_global)

            return (te, te_lower, te_upper)

    def estimate_ate(
        self,
        X,
        treatment,
        y,
        p=None,
        num_samples=5,
        bootstrap_ci=False,
        n_bootstraps=1000,
        bootstrap_size=10000,
        pretrain=False,
    ):
        """Estimate the Average Treatment Effect (ATE).

        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series): a treatment vector
            y (np.array or pd.Series): an outcome vector
            p (np.ndarray or pd.Series or dict, optional): an array of propensity scores of float (0,1) in the
                single-treatment case; or, a dictionary of treatment groups that map to propensity vectors of
                float (0,1); if None will run ElasticNetPropensityModel() to generate the propensity scores.
            num_samples (int): number of pairs for each control observation
            bootstrap_ci (bool): whether run bootstrap for confidence intervals
            n_bootstraps (int): number of bootstrap iterations
            bootstrap_size (int): number of samples per bootstrap
            pretrain (bool): whether a model has been fit, default False.
        Returns:
            The mean and confidence interval (LB, UB) of the ATE estimate.
        """
        if pretrain:
            if p is None:
                # when p is null, use pretrain propensity score
                if not self.propensity:
                    raise ValueError("no propensity score, please call fit() first")
                te, fhat = self.predict(
                    X, treatment, y, return_components=True
                )
            else:
                p = self._format_p(p, self.t_groups)
                te, fhat = self.predict(
                    X, treatment, y, return_components=True
                )
        else:
            te, fhat = self.fit_predict(
                X, treatment, y, num_samples, return_components=True
            )
        X, treatment, y = convert_pd_to_np(X, treatment, y)

        # if p is None:
        #     p = self.propensity
        # else:
        #     p = self._format_p(p, self.t_groups)

        ate = np.zeros(self.t_groups.shape[0])
        ate_lb = np.zeros(self.t_groups.shape[0])
        ate_ub = np.zeros(self.t_groups.shape[0])

        ate[0] = te[:, 0].mean()

        # for i, group in enumerate(self.t_groups):
        #     _ate = te[:, i].mean()

        #     mask = (treatment == group) | (treatment == self.control_name)
        #     treatment_filt = treatment[mask]
        #     w = (treatment_filt == group).astype(int)
        #     prob_treatment = float(sum(w)) / w.shape[0]

        #     dhat_c = dhat_cs[group][mask]
        #     dhat_t = dhat_ts[group][mask]
        #     p_filt = p[group][mask]

        #     # SE formula is based on the lower bound formula (7) from Imbens, Guido W., and Jeffrey M. Wooldridge. 2009.
        #     # "Recent Developments in the Econometrics of Program Evaluation." Journal of Economic Literature
        #     se = np.sqrt(
        #         (
        #             self.vars_t[group] / prob_treatment
        #             + self.vars_c[group] / (1 - prob_treatment)
        #             + (p_filt * dhat_c + (1 - p_filt) * dhat_t).var()
        #         )
        #         / w.shape[0]
        #     )

        #     _ate_lb = _ate - se * norm.ppf(1 - self.ate_alpha / 2)
        #     _ate_ub = _ate + se * norm.ppf(1 - self.ate_alpha / 2)

        #     ate[i] = _ate
        #     ate_lb[i] = _ate_lb
        #     ate_ub[i] = _ate_ub

        if not bootstrap_ci:
            return ate, ate_lb, ate_ub
        else:
            t_groups_global = self.t_groups
            _classes_global = self._classes
            model_mu_global = deepcopy(self.model_mu)
            model_nu_global = deepcopy(self.model_nu)

            logger.info("Bootstrap Confidence Intervals for ATE")
            ate_bootstraps = np.zeros(shape=(self.t_groups.shape[0], n_bootstraps))

            for n in tqdm(range(n_bootstraps)):
                cate_b = self.bootstrap(X, treatment, y, p, size=bootstrap_size)
                ate_bootstraps[:, n] = cate_b.mean()

            ate_lower = np.percentile(
                ate_bootstraps, (self.ate_alpha / 2) * 100, axis=1
            )
            ate_upper = np.percentile(
                ate_bootstraps, (1 - self.ate_alpha / 2) * 100, axis=1
            )

            # set member variables back to global (currently last bootstrapped outcome)
            self.t_groups = t_groups_global
            self._classes = _classes_global
            self.model_mu = deepcopy(model_mu_global)
            self.models_nu = deepcopy(model_nu_global)
            return ate, ate_lower, ate_upper

In [ ]:
class BasePHRegressor(BasePHLearner):
    """
    A parent class for PH-learner regressor classes.
    """

    def __init__(
        self,
        learner=None,
        outcome_learner=None,
        pair_learner=None,
        ate_alpha=0.05,
        control_name=0,
    ):
        """Initialize a PH-learner regressor.

        Args:
            learner (optional): a model to estimate outcomes and treatment effects in both the control and treatment
                groups
            outcome_learner (optional): a model to estimate outcomes
            pair_learner (optional): a model to estimate paired treatment effects
            ate_alpha (float, optional): the confidence level alpha of the ATE estimate
            control_name (str or int, optional): name of control group
        """
        super().__init__(
            learner=learner,
            outcome_learner=outcome_learner,
            pair_learner=pair_learner,
            ate_alpha=ate_alpha,
            control_name=control_name,
        )

In [ ]:
class BasePDRLearner(BaseLearner):
    """
    A parent class for PDR-learner regressor classes.
    PDR-learner estimates treatment effects with machine learning models.
    """

    def __init__(
        self,
        learner=None,
        control_outcome_learner=None,
        treatment_outcome_learner=None,
        treatment_effect_learner=None,
        ate_alpha=0.05,
        control_name=0,
    ):
        """
        Initialize a PDR-learner.

        Args:
            learner (optional): model used for all tasks if specific learners are not provided.
            control_outcome_learner (optional): model for control outcomes.
            treatment_outcome_learner (optional): model for treated outcomes.
            treatment_effect_learner (optional): model for treatment effects.
            ate_alpha (float, optional): significance level for ATE CI.
            control_name (str or int, optional): label for control group.
        """
        assert (learner is not None) or (
            (control_outcome_learner is not None)
            and (treatment_outcome_learner is not None)
            and (treatment_effect_learner is not None)
        )

        if control_outcome_learner is None:
            self.model_mu_c = deepcopy(learner)
        else:
            self.model_mu_c = control_outcome_learner

        if treatment_outcome_learner is None:
            self.model_mu_t = deepcopy(learner)
        else:
            self.model_mu_t = treatment_outcome_learner

        if treatment_effect_learner is None:
            self.model_tau = deepcopy(learner)
        else:
            self.model_tau = treatment_effect_learner

        self.ate_alpha = ate_alpha
        self.control_name = control_name

        self.propensity = None

    def __repr__(self):
        return (
            "{}(control_outcome_learner={},\n"
            "\ttreatment_outcome_learner={},\n"
            "\ttreatment_effect_learner={})".format(
                self.__class__.__name__,
                self.model_mu_c.__repr__(),
                self.model_mu_t.__repr__(),
                self.model_tau.__repr__(),
            )
        )

    def fit(self, X, treatment, y, p=None, seed=None):
        """
        Fit the PDR-learner with the doubly robust pairwise estimator.

        Args:
            X (np.array or pd.DataFrame): feature matrix.
            treatment (np.array or pd.Series): treatment vector.
            y (np.array or pd.Series): outcome vector.
            p (optional): propensity scores; if None, they are estimated.
            seed (int): random seed.
        """
        X, treatment, y = convert_pd_to_np(X, treatment, y)
        check_treatment_vector(treatment, self.control_name)
        self.t_groups = np.unique(treatment[treatment != self.control_name])
        self.t_groups.sort()
        self._classes = {group: i for i, group in enumerate(self.t_groups)}

        # Use 3-fold cross-fitting
        cv = KFold(n_splits=3, shuffle=True, random_state=seed)
        split_indices = [index for _, index in cv.split(y)]

        self.models_mu_c = [deepcopy(self.model_mu_c) for _ in range(3)]
        self.models_mu_t = {
            group: [deepcopy(self.model_mu_t) for _ in range(3)]
            for group in self.t_groups
        }
        self.models_tau = {
            group: [deepcopy(self.model_tau) for _ in range(3)]
            for group in self.t_groups
        }

        # Initialize propensity score container if not provided
        if p is None:
            self.propensity = {group: np.zeros(y.shape[0]) for group in self.t_groups}

        # Cross-fit
        for ifold in range(3):
            treatment_idx = split_indices[ifold]
            outcome_idx = split_indices[(ifold + 1) % 3]
            tau_idx = split_indices[(ifold + 2) % 3]

            treatment_treat = treatment[treatment_idx]
            treatment_out = treatment[outcome_idx]
            treatment_tau = treatment[tau_idx]

            y_out, y_tau = y[outcome_idx], y[tau_idx]
            X_treat, X_out, X_tau = X[treatment_idx], X[outcome_idx], X[tau_idx]

            # Propensity score estimation if not provided
            if p is None:
                logger.info("Estimating propensity scores")
                cur_p = dict()
                for group in self.t_groups:
                    mask = (treatment_treat == group) | (treatment_treat == self.control_name)
                    X_filt = X_treat[mask]
                    treatment_filt = treatment_treat[mask]
                    w_filt = (treatment_filt == group).astype(int)
                    # Compute propensity scores for group 'group'
                    cur_p[group], _ = compute_propensity_score(
                        X=X_filt, treatment=w_filt, X_pred=X_tau, treatment_pred=(treatment_tau == group).astype(int)
                    )
                    self.propensity[group][tau_idx] = cur_p[group]
            else:
                cur_p = dict()
                if isinstance(p, (np.ndarray, pd.Series)):
                    cur_p = {self.t_groups[0]: convert_pd_to_np(p[tau_idx])}
                else:
                    cur_p = {g: p[g][tau_idx] for g in self.t_groups}
                check_p_conditions(cur_p, self.t_groups)

            # Outcome regression: fit control and treatment models on outcome fold
            logger.info("Fitting outcome regressions")
            # Fit control outcome model on control units
            self.models_mu_c[ifold].fit(
                X_out[treatment_out == self.control_name],
                y_out[treatment_out == self.control_name],
            )
            for group in self.t_groups:
                # Fit treated outcome model for each group
                self.models_mu_t[group][ifold].fit(
                    X_out[treatment_out == group],
                    y_out[treatment_out == group],
                )

            # Fit pseudo outcomes for treatment effect using doubly robust pairwise estimator
            logger.info("Fitting doubly robust pairwise pseudo outcomes")
            for group in self.t_groups:
                # Filter tau-fold: keep observations from control or group 'group'
                mask = (treatment_tau == group) | (treatment_tau == self.control_name)
                X_filt = X_tau[mask]
                y_filt = y_tau[mask]
                p_filt = cur_p[group][mask]
                # Predict outcomes using outcome models
                mu_c_all = self.models_mu_c[ifold].predict(X_filt)
                mu_t_all = self.models_mu_t[group][ifold].predict(X_filt)

                # Separate into control and treated subsets for pairing
                mask_control = (treatment_tau[mask] == self.control_name)
                mask_treated = (treatment_tau[mask] == group)
                if np.sum(mask_control) == 0 or np.sum(mask_treated) == 0:
                    logger.warning("Not enough data for pairing in group {}".format(group))
                    continue

                X_control = X_filt[mask_control]
                y_control = y_filt[mask_control]
                p_control = p_filt[mask_control]
                mu_control = mu_c_all[mask_control]

                X_treated = X_filt[mask_treated]
                y_treated = y_filt[mask_treated]
                p_treated = p_filt[mask_treated]
                mu_treated = mu_t_all[mask_treated]

                scaler = MinMaxScaler()

                X_control_scaled = scaler.fit_transform(X_control)
                X_treated_scaled = scaler.fit_transform(X_treated)

                # Build KDTree on the control group
                tree = cKDTree(X_control_scaled)

                k = 10

                # Find the nearest control neighbour for each treated sample
                distances, idx_control = tree.query(X_treated_scaled, k=k)  # k=5

                # Expand treated indices to match the number of pairs (each treated unit repeats k times)
                idx_treated = np.repeat(np.arange(len(X_treated)), k)
                idx_control = idx_control.flatten()  # Flatten to align with repeated treated indices


                # Compute the doubly robust pairwise pseudo outcome for each pair:
                #   dr_pair = [mu_treated - mu_control] +
                #             [ (y_treated - mu_treated) / p_treated - (y_control - mu_control) / (1-p_control) ]
                dr_pairs = (
                    (mu_treated[idx_treated] - mu_control[idx_control])
                    + ((y_treated[idx_treated] - mu_treated[idx_treated]) / p_treated[idx_treated]
                       - (y_control[idx_control] - mu_control[idx_control]) / (1 - p_control[idx_control]))
                )
                # Combine the paired features. Here we simply concatenate the control and treated features.
                X_pairs = np.hstack([X_control[idx_control], X_treated[idx_treated]])
                # Fit the treatment effect learner on the paired data and doubly robust pseudo outcomes.
                self.models_tau[group][ifold].fit(X_pairs, dr_pairs)

    def predict(self, X, treatment=None, y=None, p=None, return_components=False, verbose=True):
        """
        Predict treatment effects.
        Args:
            X (np.array or pd.DataFrame): feature matrix.
            treatment (optional): treatment vector.
            y (optional): outcome vector.
            return_components (bool): if True, return predicted outcomes for control and treated separately.
            verbose (bool): whether to output logs.
        Returns:
            np.array: predicted treatment effects.
        """
        X, treatment, y = convert_pd_to_np(X, treatment, y)
        te = np.zeros((X.shape[0], len(self.t_groups)))
        yhat_cs = {}
        yhat_ts = {}

        for i, group in enumerate(self.t_groups):
            # Average the treatment effect predictions from the cross-fit models
            models_tau = self.models_tau[group]
            _te = np.r_[[model.predict(np.hstack([X, X])) for model in models_tau]].mean(axis=0)
            te[:, i] = np.ravel(_te)
            yhat_cs[group] = np.r_[
                [model.predict(X) for model in self.models_mu_c]
            ].mean(axis=0)
            yhat_ts[group] = np.r_[
                [model.predict(X) for model in self.models_mu_t[group]]
            ].mean(axis=0)

            if (y is not None) and (treatment is not None) and verbose:
                mask = (treatment == group) | (treatment == self.control_name)
                treatment_filt = treatment[mask]
                y_filt = y[mask]
                w = (treatment_filt == group).astype(int)
                yhat = np.zeros_like(y_filt, dtype=float)
                yhat[w == 0] = yhat_cs[group][mask][w == 0]
                yhat[w == 1] = yhat_ts[group][mask][w == 1]
                logger.info("Error metrics for group {}".format(group))
                regression_metrics(y_filt, yhat, w)
        if not return_components:
            return te
        else:
            return te, yhat_cs, yhat_ts

    def fit_predict(
        self,
        X,
        treatment,
        y,
        p=None,
        return_ci=False,
        n_bootstraps=1000,
        bootstrap_size=10000,
        return_components=False,
        verbose=True,
        seed=None,
    ):
        """Fit the treatment effect and outcome models of the R learner and predict treatment effects.

        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series): a treatment vector
            y (np.array or pd.Series): an outcome vector
            p (np.ndarray or pd.Series or dict, optional): an array of propensity scores of float (0,1) in the
                single-treatment case; or, a dictionary of treatment groups that map to propensity vectors of
                float (0,1); if None will run ElasticNetPropensityModel() to generate the propensity scores.
            return_ci (bool): whether to return confidence intervals
            n_bootstraps (int): number of bootstrap iterations
            bootstrap_size (int): number of samples per bootstrap
            return_components (bool, optional): whether to return outcome for treatment and control seperately
            verbose (str): whether to output progress logs
            seed (int): random seed for cross-fitting
        Returns:
            (numpy.ndarray): Predictions of treatment effects. Output dim: [n_samples, n_treatment]
                If return_ci, returns CATE [n_samples, n_treatment], LB [n_samples, n_treatment],
                UB [n_samples, n_treatment]
        """
        X, treatment, y = convert_pd_to_np(X, treatment, y)
        self.fit(X, treatment, y, p, seed)

        if p is None:
            p = self.propensity

        check_p_conditions(p, self.t_groups)
        if isinstance(p, (np.ndarray, pd.Series)):
            treatment_name = self.t_groups[0]
            p = {treatment_name: convert_pd_to_np(p)}
        elif isinstance(p, dict):
            p = {
                treatment_name: convert_pd_to_np(_p) for treatment_name, _p in p.items()
            }

        te = self.predict(
            X, treatment=treatment, y=y, return_components=return_components
        )

        if not return_ci:
            return te
        else:
            t_groups_global = self.t_groups
            _classes_global = self._classes
            models_mu_c_global = deepcopy(self.models_mu_c)
            models_mu_t_global = deepcopy(self.models_mu_t)
            models_tau_global = deepcopy(self.models_tau)
            te_bootstraps = np.zeros(
                shape=(X.shape[0], self.t_groups.shape[0], n_bootstraps)
            )

            logger.info("Bootstrap Confidence Intervals")
            for i in tqdm(range(n_bootstraps)):
                te_b = self.bootstrap(X, treatment, y, p, size=bootstrap_size)
                te_bootstraps[:, :, i] = te_b

            te_lower = np.percentile(te_bootstraps, (self.ate_alpha / 2) * 100, axis=2)
            te_upper = np.percentile(
                te_bootstraps, (1 - self.ate_alpha / 2) * 100, axis=2
            )

            # set member variables back to global (currently last bootstrapped outcome)
            self.t_groups = t_groups_global
            self._classes = _classes_global
            self.models_mu_c = deepcopy(models_mu_c_global)
            self.models_mu_t = deepcopy(models_mu_t_global)
            self.models_tau = deepcopy(models_tau_global)

            return (te, te_lower, te_upper)

    def estimate_ate(
        self,
        X,
        treatment,
        y,
        p=None,
        bootstrap_ci=False,
        n_bootstraps=1000,
        bootstrap_size=10000,
        seed=None,
        pretrain=False,
    ):
        """Estimate the Average Treatment Effect (ATE).

        Args:
            X (np.matrix or np.array or pd.Dataframe): a feature matrix
            treatment (np.array or pd.Series): a treatment vector
            y (np.array or pd.Series): an outcome vector
            p (np.ndarray or pd.Series or dict, optional): an array of propensity scores of float (0,1) in the
                single-treatment case; or, a dictionary of treatment groups that map to propensity vectors of
                float (0,1); if None will run ElasticNetPropensityModel() to generate the propensity scores.
            bootstrap_ci (bool): whether run bootstrap for confidence intervals
            n_bootstraps (int): number of bootstrap iterations
            bootstrap_size (int): number of samples per bootstrap
            seed (int): random seed for cross-fitting
            pretrain (bool): whether a model has been fit, default False.
        Returns:
            The mean and confidence interval (LB, UB) of the ATE estimate.
        """
        if pretrain:
            te, yhat_cs, yhat_ts = self.predict(
                X, treatment, y, p, return_components=True
            )
        else:
            te, yhat_cs, yhat_ts = self.fit_predict(
                X, treatment, y, p, return_components=True, seed=seed
            )
        X, treatment, y = convert_pd_to_np(X, treatment, y)

        if p is None:
            p = self.propensity
        else:
            check_p_conditions(p, self.t_groups)
        if isinstance(p, (np.ndarray, pd.Series)):
            treatment_name = self.t_groups[0]
            p = {treatment_name: convert_pd_to_np(p)}
        elif isinstance(p, dict):
            p = {
                treatment_name: convert_pd_to_np(_p) for treatment_name, _p in p.items()
            }

        ate = np.zeros(self.t_groups.shape[0])
        ate_lb = np.zeros(self.t_groups.shape[0])
        ate_ub = np.zeros(self.t_groups.shape[0])

        for i, group in enumerate(self.t_groups):
            _ate = te[:, i].mean()

            mask = (treatment == group) | (treatment == self.control_name)
            treatment_filt = treatment[mask]
            w = (treatment_filt == group).astype(int)
            prob_treatment = float(sum(w)) / w.shape[0]

            yhat_c = yhat_cs[group][mask]
            yhat_t = yhat_ts[group][mask]
            y_filt = y[mask]

            # SE formula is based on the lower bound formula (7) from Imbens, Guido W., and Jeffrey M. Wooldridge. 2009.
            # "Recent Developments in the Econometrics of Program Evaluation." Journal of Economic Literature
            se = np.sqrt(
                (
                    (y_filt[w == 0] - yhat_c[w == 0]).var() / (1 - prob_treatment)
                    + (y_filt[w == 1] - yhat_t[w == 1]).var() / prob_treatment
                    + (yhat_t - yhat_c).var()
                )
                / y_filt.shape[0]
            )

            _ate_lb = _ate - se * norm.ppf(1 - self.ate_alpha / 2)
            _ate_ub = _ate + se * norm.ppf(1 - self.ate_alpha / 2)

            ate[i] = _ate
            ate_lb[i] = _ate_lb
            ate_ub[i] = _ate_ub

        if not bootstrap_ci:
            return ate, ate_lb, ate_ub
        else:
            t_groups_global = self.t_groups
            _classes_global = self._classes
            models_mu_c_global = deepcopy(self.models_mu_c)
            models_mu_t_global = deepcopy(self.models_mu_t)
            models_tau_global = deepcopy(self.models_tau)

            logger.info("Bootstrap Confidence Intervals for ATE")
            ate_bootstraps = np.zeros(shape=(self.t_groups.shape[0], n_bootstraps))

            for n in tqdm(range(n_bootstraps)):
                cate_b = self.bootstrap(
                    X, treatment, y, p, size=bootstrap_size, seed=seed
                )
                ate_bootstraps[:, n] = cate_b.mean()

            ate_lower = np.percentile(
                ate_bootstraps, (self.ate_alpha / 2) * 100, axis=1
            )
            ate_upper = np.percentile(
                ate_bootstraps, (1 - self.ate_alpha / 2) * 100, axis=1
            )

            # set member variables back to global (currently last bootstrapped outcome)
            self.t_groups = t_groups_global
            self._classes = _classes_global
            self.models_mu_c = deepcopy(models_mu_c_global)
            self.models_mu_t = deepcopy(models_mu_t_global)
            self.models_tau = deepcopy(models_tau_global)
            return ate, ate_lower, ate_upper


class BasePDRRegressor(BasePDRLearner):
    """
    A parent class for PDR-learner regressor classes.
    """
    def __init__(
        self,
        learner=None,
        control_outcome_learner=None,
        treatment_outcome_learner=None,
        treatment_effect_learner=None,
        ate_alpha=0.05,
        control_name=0,
    ):
        super().__init__(
            learner=learner,
            control_outcome_learner=control_outcome_learner,
            treatment_outcome_learner=treatment_outcome_learner,
            treatment_effect_learner=treatment_effect_learner,
            ate_alpha=ate_alpha,
            control_name=control_name,
        )

In [ ]:
from scipy.spatial.distance import cdist
from scipy.special import softmax as scipy_softmax
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split # For potential validation split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

# --- Helper Function to Build MLP ---
def _build_mlp(input_dim, hidden_dims, output_dim=1, activation=nn.ReLU):
    """Builds a simple MLP."""
    layers = []
    last_dim = input_dim
    for hidden_dim in hidden_dims:
        layers.append(nn.Linear(last_dim, hidden_dim))
        layers.append(activation())
        # Maybe add Dropout here: layers.append(nn.Dropout(p=0.5))
        last_dim = hidden_dim
    layers.append(nn.Linear(last_dim, output_dim))
    return nn.Sequential(*layers)

# --- Custom Dataset for Pairs ---
class PairDataset(Dataset):
    """PyTorch Dataset to serve pairs of indices."""
    def __init__(self, pairs):
        self.pairs = pairs # List of tuples (idx_i, idx_j)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        # Return the pair of original indices
        return self.pairs[idx]

# --- PairNet Implementation with PyTorch ---
class PairNetTorch:
    """
    PairNet implementation using PyTorch for explicit pairwise loss optimization.

    Uses a T-Learner architecture (separate networks for control and treatment)
    trained jointly with the loss: ((y_i - y_j) - (mu(xi, ti) - mu(xj, tj)))^2
    """
    def __init__(self,
                 input_dim: int,
                 hidden_dims: list = [64, 32],
                 activation = nn.ReLU,
                 learning_rate: float = 1e-3,
                 weight_decay: float = 1e-4,
                 epochs: int = 50,
                 batch_size: int = 128,
                 distance_threshold: float = np.inf,
                 num_neighbours: int = 10, # k for KNN, num_samples for softmax
                 pairing_method: str = 'softmax', # 'softmax' or 'knn'
                 val_size: float = 0.0, # Set > 0 to use early stopping based on validation loss
                 patience: int = 5,     # Early stopping patience
                 verbose: int = 10,     # Print loss every `verbose` epochs
                 random_state: int | None = None,
                 device: str | None = None):
        """
        Initialize the PairNetTorch learner.

        Args:
            input_dim (int): Number of features in X.
            hidden_dims (list, optional): List of hidden layer sizes for mu_c and mu_t.
                                          Defaults to [64, 32].
            activation (torch.nn.Module, optional): Activation function. Defaults to nn.ReLU.
            learning_rate (float, optional): Optimizer learning rate. Defaults to 1e-3.
            weight_decay (float, optional): L2 regularization strength. Defaults to 1e-4.
            epochs (int, optional): Number of training epochs. Defaults to 50.
            batch_size (int, optional): Batch size for training pairs. Defaults to 128.
            distance_threshold (float, optional): Max distance for pairing. Defaults to np.inf.
            num_neighbours (int, optional): neighbours per sample in pairing. Defaults to 10.
            val_size (float, optional): Fraction of data to use for validation/early stopping.
                                        If 0, no validation/early stopping is used. Defaults to 0.0.
            patience (int, optional): Early stopping patience (epochs without improvement). Defaults to 5.
            verbose (int, optional): Print loss frequency (epochs). Defaults to 10.
            random_state (int | None, optional): Random seed. Defaults to None.
            device (str | None, optional): Device ('cpu', 'cuda'). Autodetects if None.
        """
        if random_state is not None:
            torch.manual_seed(random_state)
            np.random.seed(random_state)

        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.activation = activation
        self.lr = learning_rate
        self.weight_decay = weight_decay
        self.epochs = epochs
        self.batch_size = batch_size
        self.distance_threshold = distance_threshold
        self.num_neighbours = num_neighbours
        self.pairing_method = pairing_method
        self.val_size = val_size
        self.patience = patience
        self.verbose = verbose
        self.random_state = random_state
        self.rng = np.random.default_rng(random_state)

        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        #print(f"Using device: {self.device}")

        # Initialize models, optimizers, scaler
        self.mu_c = _build_mlp(input_dim, hidden_dims, 1, activation).to(self.device)
        self.mu_t = _build_mlp(input_dim, hidden_dims, 1, activation).to(self.device)

        # Combine parameters for a single optimizer pass
        all_params = list(self.mu_c.parameters()) + list(self.mu_t.parameters())
        self.optimizer = optim.Adam(all_params, lr=self.lr, weight_decay=self.weight_decay)

        self.scaler = StandardScaler() # Scale features for NN stability

        # Placeholders for data and results
        self.X_tensor = None
        self.t_tensor = None
        self.y_tensor = None
        self.pairs_train = []
        self.pairs_val = []
        self.best_val_loss = np.inf
        self.epochs_no_improve = 0
        self.best_mu_c_state = None
        self.best_mu_t_state = None


    def _softmax(self, x: np.ndarray) -> np.ndarray:
        """Numerically stable softmax function (using scipy)."""
        if x.size == 0: return np.array([])
        # Scipy's softmax handles potential numerical issues better
        return scipy_softmax(x)

    def _create_pairs_torch(self, X_np: np.ndarray, t_np: np.ndarray) -> list[tuple[int, int]]:
        """
        Creates pairs based on distance in the original feature space using
        either 'softmax' sampling or 'knn' selection.
        Returns a list of (idx_i, idx_j) tuples.
        """
        n_samples = X_np.shape[0]
        idx = np.arange(n_samples)
        idx_c = idx[t_np == 0]
        idx_t = idx[t_np == 1]

        if len(idx_c) == 0 or len(idx_t) == 0:
            print("Warning: No samples in control or treatment group for pairing.")
            return []

        X_c = X_np[idx_c]
        X_t = X_np[idx_t]

        all_pairs = [] # List of (original_index_i, original_index_j)

        # --- Function to process one direction (e.g., target=Treated, pool=Control) ---
        def find_pairs_one_direction(target_indices, target_X, pool_indices, pool_X):
            local_pairs = []
            if len(target_X) == 0 or len(pool_X) == 0:
                return local_pairs

            distances = cdist(target_X, pool_X, metric='euclidean')

            for i in range(len(target_X)): # For each target unit i
                dists_i = distances[i, :]
                original_target_idx = target_indices[i]

                if self.pairing_method == 'softmax':
                    # --- Softmax Sampling Logic ---
                    potential_neighbour_indices = np.where(dists_i <= self.distance_threshold)[0]
                    if len(potential_neighbour_indices) == 0: continue

                    valid_distances = dists_i[potential_neighbour_indices]
                    neg_distances = -valid_distances
                    probs = self._softmax(neg_distances)
                    if np.any(np.isnan(probs)) or not np.isclose(np.sum(probs), 1.0): probs = None

                    num_to_sample = min(self.num_neighbours, len(potential_neighbour_indices))
                    if num_to_sample == 0: continue

                    try:
                         sampled_local_indices = self.rng.choice(
                             potential_neighbour_indices, size=num_to_sample, replace=False, p=probs
                         )
                    except ValueError as e: # Catch potential issue with p summing slightly off 1.0
                         print(f"Warning: RNG choice error (likely prob sum issue) for sample {original_target_idx}: {e}. Sampling uniformly.")
                         sampled_local_indices = self.rng.choice(
                             potential_neighbour_indices, size=num_to_sample, replace=False # No p specified means uniform
                         )


                    for local_idx in sampled_local_indices:
                        original_pool_idx = pool_indices[local_idx]
                        local_pairs.append((original_target_idx, original_pool_idx))

                elif self.pairing_method == 'knn':
                    # --- KNN Logic ---
                    # Determine k: number of neighbours to consider
                    k = min(self.num_neighbours, len(pool_indices))
                    if k == 0: continue

                    # Find the indices of the k nearest neighbours in the pool
                    knn_local_indices = np.argsort(dists_i)[:k]
                    knn_distances = dists_i[knn_local_indices]

                    # Filter these k neighbours by the distance threshold
                    valid_knn_mask = knn_distances <= self.distance_threshold
                    final_knn_local_indices = knn_local_indices[valid_knn_mask]

                    # Create pairs with all valid k-nearest neighbours
                    for local_idx in final_knn_local_indices:
                        original_pool_idx = pool_indices[local_idx]
                        local_pairs.append((original_target_idx, original_pool_idx))
                else:
                    # Should not happen due to __init__ check, but good practice
                    raise ValueError(f"Unknown pairing method: {self.pairing_method}")

            return local_pairs

        # --- Find pairs for treated units (target=Treated, pool=Control) ---
        pairs_tc = find_pairs_one_direction(idx_t, X_t, idx_c, X_c)
        all_pairs.extend(pairs_tc)

        # --- Find pairs for control units (target=Control, pool=Treated) ---
        pairs_ct = find_pairs_one_direction(idx_c, X_c, idx_t, X_t)
        all_pairs.extend(pairs_ct)

        # Optional: Remove duplicate pairs (e.g., if (a, b) and (b, a) were both added)
        # Using frozenset to handle order invariance for uniqueness check
        unique_pairs_set = {frozenset(pair) for pair in all_pairs}
        # Convert back to list of tuples, preserving an arbitrary order for each unique pair
        # Note: This might slightly change the number of pairs compared to just extending lists
        final_pairs = [tuple(pair_set) for pair_set in unique_pairs_set]


        # print(f"Created {len(all_pairs)} raw pairs.") # Before deduplication
        #print(f"Created {len(final_pairs)} unique pairs for training/validation.")
        # return all_pairs # Return raw pairs if duplicates are desired
        return final_pairs # Return unique pairs

    def _get_predictions_for_pair(self, x_i, t_i, x_j, t_j):
        """Gets mu(x,t) predictions for both elements of pairs in a batch."""
        # Predict all i's and j's under both control and treatment scenarios
        pred_i_c = self.mu_c(x_i)
        pred_i_t = self.mu_t(x_i)
        pred_j_c = self.mu_c(x_j)
        pred_j_t = self.mu_t(x_j)

        # Select the correct prediction based on actual treatment
        # t_i and t_j need to have the same shape as predictions for torch.where
        # They should be [batch_size, 1]
        t_i_mask = t_i.bool() # Convert to boolean mask
        t_j_mask = t_j.bool()

        pred_i = torch.where(t_i_mask, pred_i_t, pred_i_c)
        pred_j = torch.where(t_j_mask, pred_j_t, pred_j_c)

        return pred_i, pred_j

    def _calculate_loss(self, idx_i_batch, idx_j_batch):
        """Calculates the pairwise loss for a batch of pairs."""
        # Retrieve data for the batch using indices
        x_i = self.X_tensor[idx_i_batch]
        t_i = self.t_tensor[idx_i_batch]
        y_i = self.y_tensor[idx_i_batch]

        x_j = self.X_tensor[idx_j_batch]
        t_j = self.t_tensor[idx_j_batch]
        y_j = self.y_tensor[idx_j_batch]

        # Get predictions using the T-learner structure
        pred_i, pred_j = self._get_predictions_for_pair(x_i, t_i, x_j, t_j)

        # Calculate the pairwise loss
        true_diff = y_i - y_j
        pred_diff = pred_i - pred_j
        loss = torch.mean((true_diff - pred_diff)**2) # Mean Squared Error of the differences
        return loss

    def fit(self, X: pd.DataFrame, treatment: pd.Series, y: pd.Series):
        """
        Fit the PairNetTorch model using the pairwise loss.

        Args:
            X (pd.DataFrame): Feature matrix.
            treatment (pd.Series): Treatment assignment vector (0 or 1).
            y (pd.Series): Outcome vector.
        """
        # --- 1. Preprocessing and Data Setup ---
        X, treatment, y = convert_pd_to_np(X, treatment, y)
        X_np = self.scaler.fit_transform(X).astype(np.float32)
        t_np = treatment.astype(np.float32).reshape(-1, 1) # Ensure shape [n, 1]
        y_np = y.astype(np.float32).reshape(-1, 1)       # Ensure shape [n, 1]

        self.X_tensor = torch.tensor(X_np, dtype=torch.float32).to(self.device)
        self.t_tensor = torch.tensor(t_np, dtype=torch.float32).to(self.device)
        self.y_tensor = torch.tensor(y_np, dtype=torch.float32).to(self.device)

        # --- 2. Create Pairs ---
        all_pairs = self._create_pairs_torch(X_np, t_np.flatten()) # Use original X for pairing
        if not all_pairs:
            raise ValueError("No pairs were created. Cannot train the model.")

        # --- 3. Split pairs for Train/Validation (if val_size > 0) ---
        if self.val_size > 0:
            if len(all_pairs) < 2 / self.val_size: # Ensure enough pairs for split
                 print("Warning: Not enough pairs for validation split, disabling validation.")
                 self.val_size = 0
                 self.pairs_train = all_pairs
                 self.pairs_val = []
            else:
                 pairs_train_idx, pairs_val_idx = train_test_split(
                    np.arange(len(all_pairs)), test_size=self.val_size, random_state=self.random_state
                 )
                 self.pairs_train = [all_pairs[i] for i in pairs_train_idx]
                 self.pairs_val = [all_pairs[i] for i in pairs_val_idx]
                 # print(f"Using {len(self.pairs_train)} pairs for training, {len(self.pairs_val)} for validation.")
        else:
             self.pairs_train = all_pairs
             self.pairs_val = []

        train_dataset = PairDataset(self.pairs_train)
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)

        if self.pairs_val:
            val_dataset = PairDataset(self.pairs_val)
            # Use larger batch size for validation as no gradients are needed
            val_loader = DataLoader(val_dataset, batch_size=self.batch_size * 2)


        # --- 4. Training Loop ---
        self.mu_c.train()
        self.mu_t.train()
        self.best_val_loss = np.inf
        self.epochs_no_improve = 0

        for epoch in range(self.epochs):
            epoch_train_loss = 0.0
            for i, (idx_i_batch, idx_j_batch) in enumerate(train_loader):
                # Indices need to be tensors for indexing other tensors
                idx_i_batch = idx_i_batch.long()
                idx_j_batch = idx_j_batch.long()

                self.optimizer.zero_grad()
                loss = self._calculate_loss(idx_i_batch, idx_j_batch)
                loss.backward()
                self.optimizer.step()
                epoch_train_loss += loss.item()

            avg_epoch_train_loss = epoch_train_loss / len(train_loader)

            # --- 5. Validation and Early Stopping ---
            avg_epoch_val_loss = -1 # Sentinel value if no validation
            if self.pairs_val:
                self.mu_c.eval()
                self.mu_t.eval()
                epoch_val_loss = 0.0
                with torch.no_grad():
                    for idx_i_batch, idx_j_batch in val_loader:
                        idx_i_batch = idx_i_batch.long()
                        idx_j_batch = idx_j_batch.long()
                        loss = self._calculate_loss(idx_i_batch, idx_j_batch)
                        epoch_val_loss += loss.item()
                avg_epoch_val_loss = epoch_val_loss / len(val_loader)

                if avg_epoch_val_loss < self.best_val_loss:
                    self.best_val_loss = avg_epoch_val_loss
                    self.epochs_no_improve = 0
                    # Store the best model state
                    self.best_mu_c_state = self.mu_c.state_dict()
                    self.best_mu_t_state = self.mu_t.state_dict()
                else:
                    self.epochs_no_improve += 1

                # Set back to train mode
                self.mu_c.train()
                self.mu_t.train()

            if self.verbose > 0 and (epoch + 1) % self.verbose == 0:
                if self.pairs_val:
                    print(f"Epoch [{epoch+1}/{self.epochs}], Train Loss: {avg_epoch_train_loss:.4f}, Val Loss: {avg_epoch_val_loss:.4f}")
                else:
                    print(f"Epoch [{epoch+1}/{self.epochs}], Train Loss: {avg_epoch_train_loss:.4f}")

            if self.val_size > 0 and self.epochs_no_improve >= self.patience:
                print(f"Early stopping triggered after epoch {epoch+1}. Best Val Loss: {self.best_val_loss:.4f}")
                # Load the best model state before exiting
                if self.best_mu_c_state and self.best_mu_t_state:
                    self.mu_c.load_state_dict(self.best_mu_c_state)
                    self.mu_t.load_state_dict(self.best_mu_t_state)
                break

        # If not early stopping or if early stopping happened on last epoch,
        # potentially load the best model state if validation was used.
        if self.val_size > 0 and self.best_mu_c_state and self.best_mu_t_state:
             self.mu_c.load_state_dict(self.best_mu_c_state)
             self.mu_t.load_state_dict(self.best_mu_t_state)

        return self

    def predict(self, X: pd.DataFrame, return_components: bool = False) -> np.ndarray | tuple:
        """
        Predict Conditional Average Treatment Effect (CATE).

        Args:
            X (pd.DataFrame): Feature matrix for prediction.
            return_components (bool, optional): If True, return tuple of
                                                (cate, y_pred_c, y_pred_t).
                                                Defaults to False.

        Returns:
            np.ndarray or tuple: Predicted CATE = mu_t(X) - mu_c(X)
                                 (or tuple if return_components is True).
        """
        if self.X_tensor is None: # Basic check if fit has been called
            raise RuntimeError("Model has not been fitted yet. Call fit() first.")

        self.mu_c.eval()
        self.mu_t.eval()

        X = convert_pd_to_np(X)
        X_np = self.scaler.transform(X).astype(np.float32) # Use fitted scaler
        X_pred_tensor = torch.tensor(X_np, dtype=torch.float32).to(self.device)

        with torch.no_grad():
            y_pred_c = self.mu_c(X_pred_tensor)
            y_pred_t = self.mu_t(X_pred_tensor)

        cate_pred = y_pred_t - y_pred_c

        # Convert to numpy arrays
        cate_pred_np = cate_pred.cpu().numpy()
        if return_components:
            y_pred_c_np = y_pred_c.cpu().numpy()
            y_pred_t_np = y_pred_t.cpu().numpy()
            return cate_pred_np, y_pred_c_np, y_pred_t_np
        else:
            return cate_pred_np

    def estimate_ate(self, X: pd.DataFrame) -> float:
        """
        Estimate the Average Treatment Effect (ATE).

        Args:
            X (pd.DataFrame): Feature matrix for ATE estimation.

        Returns:
            float: Estimated ATE.
        """
        cate_pred = self.predict(X, return_components=False)
        return float(np.mean(cate_pred))


In [ ]:
"""
Utilities and helpers for retrieving the datasets
"""
# stdlib
import tarfile
import urllib.request
from pathlib import Path
from typing import Optional
import os


def download_gdrive_if_needed(path: Path, file_id: str) -> None:
    """
    Helper for downloading a file from Google Drive, if it is now already on the disk.

    Parameters
    ----------
    path: Path
        Where to download the file
    file_id: str
        Google Drive File ID. Details: https://developers.google.com/drive/api/v3/about-files
    """
    path = Path(path)

    if os.path.exists(path):
        return

    gdown.download(id=file_id, output=str(path), quiet=False)


def download_http_if_needed(path: Path, url: str) -> None:
    """
    Helper for downloading a file, if it is now already on the disk.

    Parameters
    ----------
    path: Path
        Where to download the file.
    url: URL string
        HTTP URL for the dataset.
    """
    path = Path(path)

    if path.exists():
        return

    if url.lower().startswith("http"):
        urllib.request.urlretrieve(url, path)  # nosec
        return

    raise ValueError(f"Invalid url provided {url}")


def unarchive_if_needed(path: Path, output_folder: Path) -> None:
    """
    Helper for uncompressing archives. Supports .tar.gz and .tar.

    Parameters
    ----------
    path: Path
        Source archive.
    output_folder: Path
        Where to unarchive.
    """
    if str(path).endswith(".tar.gz"):
        tar = tarfile.open(path, "r:gz")
        tar.extractall(path=output_folder) # nosec
        tar.close()
    elif str(path).endswith(".tar"):
        tar = tarfile.open(path, "r:")
        tar.extractall(path=output_folder) # nosec
        tar.close()
    else:
        raise NotImplementedError(f"archive not supported {path}")


def download_if_needed(
    download_path: Path,
    file_id: Optional[str] = None,  # used for downloading from Google Drive
    http_url: Optional[str] = None,  # used for downloading from a HTTP URL
    unarchive: bool = False,  # unzip a downloaded archive
    unarchive_folder: Optional[Path] = None,  # unzip folder
) -> None:
    """
    Helper for retrieving online datasets.

    Parameters
    ----------
    download_path: str
        Where to download the archive
    file_id: str, optional
        Set this if you want to download from a public Google drive share
    http_url: str, optional
        Set this if you want to download from a HTTP URL
    unarchive: bool
        Set this if you want to try to unarchive the downloaded file
    unarchive_folder: str
        Mandatory if you set unarchive to True.
    """
    download_path = Path(download_path)
    if file_id is not None:
        download_gdrive_if_needed(download_path, file_id)
    elif http_url is not None:
        download_http_if_needed(download_path, http_url)
    else:
        raise ValueError("Please provide a download URL")

    if unarchive and unarchive_folder is None:
        raise ValueError("Please provide a folder for the archive")
    if unarchive and unarchive_folder is not None:
        try:
            unarchive_if_needed(download_path, unarchive_folder)
        except BaseException as e:
            print(f"Failed to unpack {download_path}. Error {e}")
            download_path.unlink()

In [ ]:
"""
IHDP (Infant Health and Development Program) dataset
"""
# stdlib
import os
import random
from pathlib import Path
from typing import Any, Tuple

# third party
import numpy as np

np.random.seed(0)
random.seed(0)

TRAIN_DATASET = "ihdp_npci_1-100.train.npz"
TEST_DATASET = "ihdp_npci_1-100.test.npz"
TRAIN_URL = "https://www.fredjo.com/files/ihdp_npci_1-100.train.npz"
TEST_URL = "https://www.fredjo.com/files/ihdp_npci_1-100.test.npz"


# helper functions
def load_data_npz(fname: Path, get_po: bool = True) -> dict:
    """
    Helper function for loading the IHDP data set (adapted from https://github.com/clinicalml/cfrnet)

    Parameters
    ----------
    fname: Path
        Dataset path

    Returns
    -------
    data: dict
        Raw IHDP dict, with X, w, y and yf keys.
    """
    data_in = np.load(fname)
    data = {"X": data_in["x"], "w": data_in["t"], "y": data_in["yf"]}
    try:
        data["ycf"] = data_in["ycf"]
    except BaseException:
        data["ycf"] = None

    if get_po:
        data["mu0"] = data_in["mu0"]
        data["mu1"] = data_in["mu1"]

    data["HAVE_TRUTH"] = not data["ycf"] is None
    data["dim"] = data["X"].shape[1]
    data["n"] = data["X"].shape[0]

    return data


def prepare_ihdp_data(
    data_train: dict,
    data_test: dict,
    rescale: bool = False,
    setting: str = "C",
    return_pos: bool = False,
) -> Tuple:
    """
    Helper for preprocessing the IHDP dataset.

    Parameters
    ----------
    data_train: pd.DataFrame or dict
        Train dataset
    data_test: pd.DataFrame or dict
        Test dataset
    rescale: bool, default False
        Rescale the outcomes to have similar scale
    setting: str, default C
        Experiment setting
    return_pos: bool
        Return potential outcomes

    Returns
    -------
    X: dict or pd.DataFrame
        Training Feature set
    y: pd.DataFrame or list
        Outcome list
    t: pd.DataFrame or list
        Treatment list
    cate_true_in: pd.DataFrame or list
        Average treatment effects for the training set
    X_t: pd.Dataframe or list
        Test feature set
    cate_true_out: pd.DataFrame of list
        Average treatment effects for the testing set
    """

    X, y, w, mu0, mu1 = (
        data_train["X"],
        data_train["y"],
        data_train["w"],
        data_train["mu0"],
        data_train["mu1"],
    )

    X_t, _, w_t, mu0_t, mu1_t = (
        data_test["X"],
        data_test["y"],
        data_test["w"],
        data_test["mu0"],
        data_test["mu1"],
    )
    if setting == "D":
        y[w == 1] = y[w == 1] + mu0[w == 1]
        mu1 = mu0 + mu1
        mu1_t = mu0_t + mu1_t

    if rescale:
        # rescale all outcomes to have similar scale of CATEs if sd_cate > 1
        cate_in = mu0 - mu1
        sd_cate = np.sqrt(cate_in.var())

        if sd_cate > 1:
            # training data
            error = y - w * mu1 - (1 - w) * mu0
            mu0 = mu0 / sd_cate
            mu1 = mu1 / sd_cate
            y = w * mu1 + (1 - w) * mu0 + error

            # test data
            mu0_t = mu0_t / sd_cate
            mu1_t = mu1_t / sd_cate

    cate_true_in = mu1 - mu0
    cate_true_out = mu1_t - mu0_t

    if return_pos:
        return X, y, w, cate_true_in, X_t, w_t, cate_true_out, mu0, mu1, mu0_t, mu1_t

    else:
        return X, y, w, cate_true_in, X_t, w_t, cate_true_out


def prepare_ihdp_pairnet_data(
    i_exp,
    model_name: str,
    data_train: dict,
    data_test: dict,
    rescale: bool = False,
    setting: str = "C",
    return_pos: bool = False,
    **kwargs,
):
    X, y, beta, cate_true_in, X_test, beta_test, cate_true_out = prepare_ihdp_data(
        data_train,
        data_test,
        rescale=rescale,
        setting=setting,
        return_pos=return_pos,
    )


    ads_train = None


    return (
        {
            "X": X,
            "y": y,
            "w": beta,
            "cate_true_in": cate_true_in,
            "X_t": X_test,
            "w_t": beta_test,
            "cate_true_out": cate_true_out,
        },
        ads_train,
    )


def get_one_data_set(D: dict, i_exp: int, get_po: bool = True) -> dict:
    """
    Helper for getting the IHDP data for one experiment. Adapted from https://github.com/clinicalml/cfrnet

    Parameters
    ----------
    D: dict or pd.DataFrame
        All the experiment
    i_exp: int
        Experiment number

    Returns
    -------
    data: dict or pd.Dataframe
        dict with the experiment
    """
    D_exp = {}
    D_exp["X"] = D["X"][:, :, i_exp - 1]
    D_exp["w"] = D["w"][:, i_exp - 1 : i_exp]
    D_exp["y"] = D["y"][:, i_exp - 1 : i_exp]
    if D["HAVE_TRUTH"]:
        D_exp["ycf"] = D["ycf"][:, i_exp - 1 : i_exp]
    else:
        D_exp["ycf"] = None

    if get_po:
        D_exp["mu0"] = D["mu0"][:, i_exp - 1 : i_exp]
        D_exp["mu1"] = D["mu1"][:, i_exp - 1 : i_exp]

    return D_exp


def load(data_path: Path, exp: int = 1, rescale: bool = False, **kwargs: Any) -> Tuple:
    """
    Get IHDP train/test datasets with treatments and labels.

    Parameters
    ----------
    data_path: Path
        Path to the dataset csv. If the data is missing, it will be downloaded.


    Returns
    -------
    X: pd.Dataframe or array
        The training feature set
    w: pd.DataFrame or array
        Training treatment assignments.
    y: pd.Dataframe or array
        The training labels
    training potential outcomes: pd.DataFrame or array.
        Potential outcomes for the training set.
    X_t: pd.DataFrame or array
        The testing feature set
    testing potential outcomes: pd.DataFrame of array
        Potential outcomes for the testing set.
    """
    data_train, data_test = load_raw(data_path)

    data_exp = get_one_data_set(data_train, i_exp=exp, get_po=True)
    data_exp_test = get_one_data_set(data_test, i_exp=exp, get_po=True)

    (
        X,
        y,
        w,
        cate_true_in,
        X_t,
        w_t,
        cate_true_out,
        mu0,
        mu1,
        mu0_t,
        mu1_t,
    ) = prepare_ihdp_data(
        data_exp,
        data_exp_test,
        rescale=rescale,
        return_pos=True,
    )

    return (
        X,
        w,
        y,
        np.asarray([mu0, mu1]).squeeze().T,
        X_t,
        np.asarray([mu0_t, mu1_t]).squeeze().T,
    )


def load_raw(data_path: Path) -> Tuple:
    """
    Get IHDP raw train/test sets.

    Parameters
    ----------
    data_path: Path
        Path to the dataset csv. If the data is missing, it will be downloaded.

    Returns
    -------

    data_train: dict or pd.DataFrame
        Training data
    data_test: dict or pd.DataFrame
        Testing data
    """

    try:
        os.mkdir(data_path)
    except BaseException:
        pass

    train_csv = data_path / TRAIN_DATASET
    test_csv = data_path / TEST_DATASET

    download_if_needed(train_csv, http_url=TRAIN_URL)
    download_if_needed(test_csv, http_url=TEST_URL)

    data_train = load_data_npz(train_csv, get_po=True)
    data_test = load_data_npz(test_csv, get_po=True)

    return data_train, data_test

In [ ]:
df = pd.DataFrame()
for i in range(1, 11):
    data = pd.read_csv('IHDP/csv/ihdp_npci_' + str(i) + '.csv', header=None)
    df = pd.concat([data, df])
cols =  ["treatment", "y_factual", "y_cfactual", "mu0", "mu1"] + [i for i in range(25)]
df.columns = cols
print(df.shape)

# replicate the data 100 times
# replications = 100
# df = pd.concat([df]*replications, ignore_index=True)
# print(df.shape)

In [ ]:
pd.Series(df['treatment']).value_counts(normalize=False)

In [ ]:
# set which features are binary
binfeats = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
# set which features are continuous
contfeats = [i for i in range(25) if i not in binfeats]

# reorder features with binary first and continuous after
perm = binfeats + contfeats

In [ ]:
df = df.reset_index(drop=True)
df.head()

In [ ]:
X = df[perm].values
treatment = df['treatment'].values
y = df['y_factual'].values
y_cf = df['y_cfactual'].values
tau = df.apply(lambda d: d['y_factual'] - d['y_cfactual'] if d['treatment']==1
               else d['y_cfactual'] - d['y_factual'],
               axis=1)
mu_0 = df['mu0'].values
mu_1 = df['mu1'].values

In [ ]:
data_train, data_test = load_raw(Path("IHDP/"))
X_train, y_train, treatment_train, tau_train, X_val, treatment_val, tau_val = prepare_ihdp_data(data_train, data_test)

In [ ]:
X_train = X_train.reshape(-1, X_train.shape[1])
y_train = y_train.reshape(-1)
treatment_train = treatment_train.reshape(-1)
tau_train = tau_train.reshape(-1)
X_val = X_val.reshape(-1, X_val.shape[1])
treatment_val = treatment_val.reshape(-1)
tau_val = tau_val.reshape(-1)

In [ ]:
# separate for train and test
# itr, ite = train_test_split(np.arange(X.shape[0]), test_size=0.2, random_state=1)
# X_train, treatment_train, y_train, y_cf_train, tau_train, mu_0_train, mu_1_train = X[itr], treatment[itr], y[itr], y_cf[itr], tau[itr], mu_0[itr], mu_1[itr]
# X_val, treatment_val, y_val, y_cf_val, tau_val, mu_0_val, mu_1_val = X[ite], treatment[ite], y[ite], y_cf[ite], tau[ite], mu_0[ite], mu_1[ite]

In [ ]:
# fit propensity model
p_model = LogisticRegression()
p_train = p_model.fit(X_train, treatment_train).predict_proba(X_train)[:,1]
p_val = p_model.fit(X_val, treatment_val).predict_proba(X_val)[:,1]

In [ ]:
def create_pairnet_nn(input_dim, hidden_dims, epochs):
    pairnet_nn = PairNetTorch(
        input_dim=input_dim,
        hidden_dims=hidden_dims, # Example architecture
        learning_rate=0.001,
        weight_decay=0.0001,
        epochs=epochs, # More epochs often needed
        batch_size=256, # Larger batch size can be more stable
        distance_threshold=2.0, # Adjust based on data scaling/distribution
        num_neighbours=3,
        pairing_method='knn',
        val_size=0.15, # Use 15% of pairs for validation/early stopping
        patience=10,
        verbose=epochs+1,
    )
    return pairnet_nn

In [ ]:
n = X_train.shape[0]
nb = 1
ci = False

learner = RandomForestRegressor(max_depth=10, n_estimators=100)
learner = MLPRegressor((16,16,16,16), max_iter=1000, early_stopping=True, )
#learner = xgb.XGBRegressor(max_depth=5)
learner = Lasso()

s_learner = BaseSRegressor(learner)
s_ate, s_ate_lb, s_ate_ub = s_learner.estimate_ate(X_train, treatment_train, y_train, return_ci=True)
s_ate, s_ate_lb, s_ate_ub = s_ate[0], s_ate_lb[0], s_ate_ub[0]
s_ite_train, s_ite_lb, s_ite_ub = s_learner.fit_predict(X_train, treatment_train, y_train, 
                                                        return_ci=ci, bootstrap_size=n, n_bootstraps=nb), '', ''
s_ite_val = s_learner.predict(X_val)

t_learner = BaseTRegressor(learner)
t_ate, t_ate_lb, t_ate_ub = t_learner.estimate_ate(X_train, treatment_train, y_train)
t_ate, t_ate_lb, t_ate_ub = t_ate[0], t_ate_lb[0], t_ate_ub[0]
t_ite_train, t_ite_lb, t_ite_ub = t_learner.fit_predict(X_train, treatment_train, y_train, 
                                                        return_ci=ci, bootstrap_size=n, n_bootstraps=nb), '', ''
t_ite_val = t_learner.predict(X_val)

x_learner = BaseXRegressor(learner)
x_ate, x_ate_lb, x_ate_ub = x_learner.estimate_ate(X_train, treatment_train, y_train, p_train)
x_ate, x_ate_lb, x_ate_ub = x_ate[0], x_ate_lb[0], x_ate_ub[0]
x_ite_train, x_ite_lb, x_ite_ub = x_learner.fit_predict(X_train, treatment_train, y_train, p_train, 
                                                        return_ci=ci, bootstrap_size=n, n_bootstraps=nb), '', ''
x_ite_val = x_learner.predict(X_val, p=p_val)

dr_learner = BaseDRRegressor(learner=learner)
dr_ate, dr_ate_lb, dr_ate_ub = dr_learner.estimate_ate(X_train, treatment_train, y_train, p_train)
dr_ate, dr_ate_lb, dr_ate_ub = dr_ate[0], dr_ate_lb[0], dr_ate_ub[0]
dr_ite_train, dr_ite_lb, dr_ite_ub = dr_learner.fit_predict(X_train, treatment_train, y_train, p_train,
                                                        return_ci=ci, bootstrap_size=n, n_bootstraps=nb), '', ''
dr_ite_val = dr_learner.predict(X_val, p=p_val)

h_learner = BaseHRegressor(outcome_learner=Lasso(), 
                           learner=learner)
h_ate, h_ate_lb, h_ate_ub = h_learner.estimate_ate(X_train, treatment_train, y_train, p_train)
h_ate, h_ate_lb, h_ate_ub = h_ate[0], h_ate_lb[0], h_ate_ub[0]
h_ite_train, h_ite_lb, h_ite_ub = h_learner.fit_predict(X_train, treatment_train, y_train, p_train, 
                                                        return_ci=ci, bootstrap_size=n, n_bootstraps=nb), '', ''
h_ite_val = h_learner.predict(X_val, p=p_val)

p_learner = BasePRegressor(learner=learner)
p_ate, p_ate_lb, p_ate_ub = p_learner.estimate_ate(X_train, treatment_train, y_train, p_train)
p_ate, p_ate_lb, p_ate_ub = p_ate[0], p_ate_lb[0], p_ate_ub[0]
p_ite_train, p_ite_lb, p_ite_ub = p_learner.fit_predict(X_train, treatment_train, y_train, p_train, num_samples=10, 
                                                        return_ci=ci, bootstrap_size=n, n_bootstraps=nb), '', ''
p_ite_val = p_learner.predict(X_val, p=p_val)

ph_learner = BasePHRegressor(outcome_learner=Lasso(), 
                             learner=learner)
ph_ate, ph_ate_lb, ph_ate_ub = ph_learner.estimate_ate(X_train, treatment_train, y_train)
ph_ate, ph_ate_lb, ph_ate_ub = ph_ate[0], ph_ate_lb[0], ph_ate_ub[0]
ph_ite_train, ph_ite_lb, ph_ite_ub = ph_learner.fit_predict(X_train, treatment_train, y_train, num_samples=10, 
                                                        return_ci=ci, bootstrap_size=n, n_bootstraps=nb), '', ''
ph_ite_val = ph_learner.predict(X_val)

pdr_learner = create_pairnet_nn(X_train.shape[1], (1,1), 1)
pdr_learner.fit(X_train, treatment_train, y_train)
pdr_ate = pdr_learner.estimate_ate(X_train)
pdr_ite_train, pdr_ite_lb, pdr_ite_ub = pdr_learner.predict(X_train), '', ''
pdr_ite_val = pdr_learner.predict(X_val)

: 

In [ ]:
print((s_ate_ub - s_ate_lb), 'S-learner')
print((t_ate_ub - t_ate_lb), 'T-learner')
print((x_ate_ub - x_ate_lb), 'X-learner')
print((dr_ate_ub - dr_ate_lb), 'DR-learner')
print((h_ate_ub - h_ate_lb), 'RT-learner')
print((p_ate_ub - p_ate_lb), 'P-learner')
print((ph_ate_ub - ph_ate_lb), 'RP-learner')

In [ ]:
df_preds_train = pd.DataFrame([s_ite_train.ravel(),
                               t_ite_train.ravel(),
                               x_ite_train.ravel(),
                               dr_ite_train.ravel(),
                               h_ite_train.ravel(),
                               p_ite_train.ravel(),
                               ph_ite_train.ravel(),
                               pdr_ite_train.ravel(),
                               tau_train.ravel(),
                               treatment_train.ravel(),
                               y_train.ravel()],
                               index=['S','T','X','DR','RT','P','RP','PDR','tau','w','y']).T


df_cumgain_train = get_cumgain(df_preds_train)

In [ ]:
df_result_train = pd.DataFrame([s_ate, t_ate, x_ate, dr_ate, h_ate, p_ate, ph_ate, pdr_ate, tau_train.mean()],
                               index=['S','T','X','DR','RT','P','RP','PDR','actual'], columns=['ATE'])
df_result_train['MSE'] = [mean_squared_error(t,p) for t,p in zip([s_ite_train, t_ite_train, x_ite_train, dr_ite_train,
                                                                  h_ite_train, p_ite_train, ph_ite_train, pdr_ite_train],
                                                                  [tau_train.reshape(-1,1)]*8)
                          ] + [None]
df_result_train['AUUC'] = auuc_score(df_preds_train)

In [ ]:
df_result_train

In [ ]:
plot_gain(df_preds_train)

In [ ]:
df_preds_val = pd.DataFrame([s_ite_val.ravel(),
                             t_ite_val.ravel(),
                             x_ite_val.ravel(),
                             dr_ite_val.ravel(),
                             h_ite_val.ravel(),
                             p_ite_val.ravel(),
                             ph_ite_val.ravel(),
                             pdr_ite_val.ravel(),
                             tau_val.ravel(),
                             treatment_val.ravel(),
                             y_val.ravel()],
                             index=['S','T','X','DR','RT','P','RP','PDR','tau','w','y']).T

df_cumgain_val = get_cumgain(df_preds_val)

In [ ]:
df_result_val = pd.DataFrame([s_ite_val.mean(), t_ite_val.mean(), x_ite_val.mean(), dr_ite_val.mean(),
                              h_ite_val.mean(), p_ite_val.mean(), ph_ite_val.mean(), pdr_ite_val.mean(), tau_val.mean()],
                              index=['S','T','X','DR','RT','P','RP','PDR','actual'], columns=['ATE'])
df_result_val['MSE'] = [mean_squared_error(t,p) for t,p in zip([s_ite_val, t_ite_val, x_ite_val, dr_ite_val,
                                                                h_ite_val, p_ite_val, ph_ite_val, pdr_ite_val],
                                                                  [tau_val.values.reshape(-1,1)]*8)
                          ] + [None]
df_result_val['AUUC'] = auuc_score(df_preds_val)

In [ ]:
df_result_val

In [ ]:
plot_gain(df_preds_val)